In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Create output directory for plots
os.makedirs('memory_analysis_plots', exist_ok=True)

# Load the CSV data
df = pd.read_csv('memory_gate_system.csv')

In [5]:
df['timestamp'] = pd.to_numeric(df['timestamp'])
df

,tag,timestamp,event,free_heap,min_free_heap,total_allocated_bytes,total_free_bytes,largest_free_block
0,MEMLOG,465,Before WiFi init,384844,384536,15388,384844,352256
1,MEMLOG,1895,After WiFi init,333992,331108,65624,333992,303104
2,MEMLOG,1935,Before Servo init,333992,331108,65624,333992,303104
3,MEMLOG,1975,After Servo init,333992,331108,65624,333992,303104
4,MEMLOG,2015,Before MQTT init,334028,331108,65592,334028,303104
5,MEMLOG,2075,After MQTT init,323352,323136,76092,323352,294912
6,MEMLOG,11505,Before open entry gate,328060,320736,71412,328060,294912
7,MEMLOG,16555,After close entry gate,325632,320736,73832,325632,294912
8,MEMLOG,19805,Before open entry gate,328060,320736,71412,328060,294912
9,MEMLOG,24845,After close entry gate,325632,320736,73832,325632,294912


In [6]:
# 1. Memory Usage Timeline
plt.figure(figsize=(14, 8))
plt.plot(df['timestamp'], df['free_heap'], marker='o', linestyle='-', color='blue')
plt.title('Memory Usage Timeline', fontsize=16)
plt.xlabel('Time (ms)', fontsize=12)
plt.ylabel('Free Heap (bytes)', fontsize=12)
plt.grid(True, alpha=0.3)

# Improve annotations for key events - offset them to avoid overlap
for idx, row in df.iterrows():
    # Alternate text positions to avoid overlap
    offset_y = 5000 if idx % 2 == 0 else -8000
    offset_x = 0
    
    # Adjust specific events that might overlap
    if 'init' in row['event']:
        offset_y = 5000 * ((idx % 3) - 1)  # -5000, 0, or 5000
    
    plt.annotate(row['event'], 
                (row['timestamp'], row['free_heap']),
                textcoords="offset points",
                xytext=(offset_x, offset_y / 1000),  # Divide by 1000 for points
                ha='center',
                fontsize=8,
                arrowprops=dict(arrowstyle="->", alpha=0.5),
                bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.7))

# Add a horizontal line for the minimum free heap
plt.axhline(y=df['min_free_heap'].iloc[0], color='r', linestyle='--', alpha=0.7)
plt.text(df['timestamp'].max() * 0.9, df['min_free_heap'].iloc[0] * 1.01, 
        f"Min Free Heap: {df['min_free_heap'].iloc[0]} bytes", 
        color='r', fontsize=10)

plt.tight_layout()
plt.savefig('memory_analysis_plots/memory_timeline.png', dpi=300)
plt.close()

In [7]:
# 2. Memory Impact by Operation (Bar Chart)
# Create pairs of operations
operations = []
memory_impacts = []

# Process WiFi Init
if 'Before WiFi init' in df['event'].values and 'After WiFi init' in df['event'].values:
    before_wifi = df[df['event'] == 'Before WiFi init']['free_heap'].iloc[0]
    after_wifi = df[df['event'] == 'After WiFi init']['free_heap'].iloc[0]
    operations.append('WiFi Init')
    memory_impacts.append(before_wifi - after_wifi)

# Process Servo Init
if 'Before Servo init' in df['event'].values and 'After Servo init' in df['event'].values:
    before_servo = df[df['event'] == 'Before Servo init']['free_heap'].iloc[0]
    after_servo = df[df['event'] == 'After Servo init']['free_heap'].iloc[0]
    operations.append('Servo Init')
    memory_impacts.append(before_servo - after_servo)

# Process MQTT Init
if 'Before MQTT init' in df['event'].values and 'After MQTT init' in df['event'].values:
    before_mqtt = df[df['event'] == 'Before MQTT init']['free_heap'].iloc[0]
    after_mqtt = df[df['event'] == 'After MQTT init']['free_heap'].iloc[0]
    operations.append('MQTT Init')
    memory_impacts.append(before_mqtt - after_mqtt)

# Process Gate Operations
if 'Before open entry gate' in df['event'].values and 'After close entry gate' in df['event'].values:
    before_gate = df[df['event'] == 'Before open entry gate']['free_heap'].mean()
    after_gate = df[df['event'] == 'After close entry gate']['free_heap'].mean()
    operations.append('Gate Operation')
    memory_impacts.append(before_gate - after_gate)

# Create bar chart
plt.figure(figsize=(12, 7))
bars = plt.bar(operations, memory_impacts, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'])
plt.title('Memory Impact by Operation', fontsize=16)
plt.xlabel('Operation', fontsize=12)
plt.ylabel('Memory Impact (bytes)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0, ha='center')  # Horizontal labels

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 200,
            f'{int(height):,}',
            ha='center', va='bottom', rotation=0, fontsize=10)

plt.tight_layout()
plt.savefig('memory_analysis_plots/memory_impact_by_operation.png', dpi=300)
plt.close()

In [8]:
# 3. Memory Recovery Pattern
# Extract gate operations
gate_before = df[df['event'] == 'Before open entry gate'].reset_index(drop=True)
gate_after = df[df['event'] == 'After close entry gate'].reset_index(drop=True)

# Ensure same number of operations
min_ops = min(len(gate_before), len(gate_after))
gate_before = gate_before.iloc[:min_ops]
gate_after = gate_after.iloc[:min_ops]

# Create operation numbers
operation_numbers = list(range(1, min_ops + 1))

plt.figure(figsize=(14, 7))
plt.plot(operation_numbers, gate_before['free_heap'], 
        marker='o', linestyle='-', label='Before Operation', color='#3498db', linewidth=2)
plt.plot(operation_numbers, gate_after['free_heap'], 
        marker='x', linestyle='--', label='After Operation', color='#e74c3c', linewidth=2)

# Fill area between the lines to highlight the difference
plt.fill_between(operation_numbers, gate_before['free_heap'], gate_after['free_heap'], 
                alpha=0.2, color='#e74c3c', label='Memory Used')

plt.title('Memory Recovery Pattern Across Gate Operations', fontsize=16)
plt.xlabel('Operation Number', fontsize=12)
plt.ylabel('Free Heap (bytes)', fontsize=12)
plt.legend(fontsize=10, loc='upper right')
plt.grid(True, alpha=0.3)
plt.ylim(min(gate_after['free_heap']) * 0.99, max(gate_before['free_heap']) * 1.01)  # Better y-axis scale
plt.tight_layout()
plt.savefig('memory_analysis_plots/memory_recovery_pattern.png', dpi=300)
plt.close()

In [9]:
# 4. Allocated vs. Free Memory
plt.figure(figsize=(14, 7))
plt.stackplot(df['timestamp'], 
            df['total_allocated_bytes'], 
            df['free_heap'],
            labels=['Allocated Memory', 'Free Memory'],
            colors=['#e74c3c', '#3498db'])

# Add markers for key events
for idx, row in df.iterrows():
    if 'init' in row['event'] or idx % 3 == 0:  # Add markers for init events and every 3rd point
        plt.scatter(row['timestamp'], row['total_allocated_bytes'] + row['free_heap']/2, 
                s=50, color='black', zorder=5)

plt.title('Memory Composition Over Time', fontsize=16)
plt.xlabel('Time (ms)', fontsize=12)
plt.ylabel('Memory (bytes)', fontsize=12)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('memory_analysis_plots/allocated_vs_free_memory.png', dpi=300)
plt.close()

In [11]:
# 5. Memory Fragmentation Analysis
plt.figure(figsize=(14, 7))
plt.plot(df['timestamp'], df['free_heap'], marker='', linestyle='-', 
        label='Total Free Bytes', color='#3498db', linewidth=2.5)
plt.plot(df['timestamp'], df['largest_free_block'], marker='', linestyle='-', 
        label='Largest Free Block', color='#f39c12', linewidth=2.5)
plt.fill_between(df['timestamp'], df['free_heap'], df['largest_free_block'], 
                alpha=0.3, color='#e74c3c', label='Fragmentation')

# Calculate and display fragmentation percentage
df['fragmentation_percent'] = (1 - (df['largest_free_block'] / df['free_heap'])) * 100

# Add markers for key events with labels for important ones
for idx, row in df.iterrows():
    if 'WiFi init' in row['event'] or 'MQTT init' in row['event']:
        plt.scatter(row['timestamp'], row['free_heap'], s=80, color='black', zorder=5)
        plt.annotate(row['event'].replace('Before ', '').replace('After ', ''), 
                    (row['timestamp'], row['free_heap']),
                    textcoords="offset points",
                    xytext=(0, 10),
                    ha='center',
                    fontsize=9,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.7))

plt.title('Memory Fragmentation Analysis', fontsize=16)
plt.xlabel('Time (ms)', fontsize=12)
plt.ylabel('Memory (bytes)', fontsize=12)
plt.legend(fontsize=10, loc='upper right')
plt.grid(True, alpha=0.3)

# Add average fragmentation text in a better position
avg_frag = df['fragmentation_percent'].mean()
plt.figtext(0.5, 0.02, f'Average Fragmentation: {avg_frag:.2f}%', 
            ha='center', fontsize=12, 
            bbox=dict(boxstyle="round,pad=0.5", fc="white", ec="#3498db", alpha=0.9))

plt.tight_layout(rect=[0, 0.03, 1, 0.97])  # Make room for the text at the bottom
plt.savefig('memory_analysis_plots/memory_fragmentation.png', dpi=300)
plt.close()

In [12]:
# 6. Gate Operation Memory Delta
memory_used = []
for i in range(min_ops):
    before_mem = gate_before['free_heap'].iloc[i]
    after_mem = gate_after['free_heap'].iloc[i]
    memory_used.append(before_mem - after_mem)

plt.figure(figsize=(12, 7))
bars = plt.bar(operation_numbers, memory_used, color='#9b59b6')
plt.title('Memory Consumption per Gate Operation', fontsize=16)
plt.xlabel('Operation Number', fontsize=12)
plt.ylabel('Memory Used (bytes)', fontsize=12)
plt.grid(axis='y', alpha=0.3)

# Add average line with improved styling
avg_mem_used = sum(memory_used) / len(memory_used)
plt.axhline(y=avg_mem_used, color='#e74c3c', linestyle='--', linewidth=2)
plt.text(len(operation_numbers) * 0.85, avg_mem_used * 1.05, 
        f'Average: {avg_mem_used:.2f} bytes', 
        color='#e74c3c', fontsize=11, 
        bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
            f'{int(height):,}',
            ha='center', va='bottom', rotation=0, fontsize=10)

# Improve x-axis ticks
plt.xticks(operation_numbers)
plt.tight_layout()
plt.savefig('memory_analysis_plots/gate_operation_memory_delta.png', dpi=300)
plt.close()

In [18]:
# 7. Memory Usage Pie Chart - For a specific point (e.g., after initialization)
if 'After MQTT init' in df['event'].values:
    final_init = df[df['event'] == 'After MQTT init'].iloc[0]
    
    plt.figure(figsize=(10, 8))
    
    # Format the values for display on the chart
    allocated = final_init['total_allocated_bytes']
    free = final_init['free_heap']
    
    # Create pie chart with improved styling
    wedges, texts, autotexts = plt.pie(
        [allocated, free], 
        labels=['Allocated Memory', 'Free Memory'],
        autopct='%1.1f%%',
        startangle=90,
        colors=['#e74c3c', '#3498db'],
        explode=(0.05, 0),
        shadow=True,
        textprops={'fontsize': 12}
    )
    
    # Customize text appearance
    for autotext in autotexts:
        autotext.set_fontsize(11)
        autotext.set_weight('bold')
        autotext.set_color('white')
    
    # Add memory values as text below the chart
    plt.figtext(0.5, 0.01, 
                f"Allocated: {allocated:,} bytes ({allocated/1024:.1f} KB)\n"
                f"Free: {free:,} bytes ({free/1024:.1f} KB)",
                ha='center', fontsize=11)
    
    plt.title('Memory Distribution After System Initialization', fontsize=16)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.tight_layout()
    plt.savefig('memory_analysis_plots/memory_distribution_pie.png', dpi=300)
    plt.close()

print("Analysis complete! Generated plots are saved in the 'memory_analysis_plots' directory.")

# Save summary tables to CSV for further use
init_summary.to_csv('memory_analysis_plots/initialization_summary.csv', index=False)
operation_summary.to_csv('memory_analysis_plots/operation_summary.csv', index=False)
frag_summary.to_csv('memory_analysis_plots/fragmentation_summary.csv', index=False)

Analysis complete! Generated plots are saved in the 'memory_analysis_plots' directory.
